### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9761920121029371495
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5012691065792644854
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#!ls /content/drive/MyDrive/'Musical 2'/projeto/dataset

In [ ]:
import os
import music21 as m21
import json
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
#us = m21.environment.UserSettings()
#us['musicxmlPath'] = r"C:\Program Files\MuseScore 3\bin\MuseScore3.exe"
#us['musescoreDirectPNGPath'] = r"C:\Program Files\MuseScore 3\bin\MuseScore3.exe"

### Preprocessamento dos Dados
* Dados do formato  .krn (partituras)
* Criar dataset a partir dessas partituras
* Converter-los para uma representação viavel para a Rede Neural

In [ ]:
# variaveis globais
PATH_TEST = 'deutschl/test'
PATH_erk = '/content/drive/MyDrive/''Musical 2''/projeto/erk'
SAVE_DIR = '/content/drive/MyDrive/''Musical 2''/projeto/dataset'
SINGLE_FILE_DATASET = '/content/drive/MyDrive/''Musical 2''/projeto/file_dataset'
SEQUENCE_LENGTH = 64
MAPPING_PATH = '/content/drive/MyDrive/''Musical 2''/projeto/mapping.json'

ACCEPTABLE_DURATIONS = [
    0.25, 
    0.5, 
    0.75,
    1.0,
    1.5,
    2.0,
    3.0,
    4.0
]

In [ ]:
# função para carregar songs do dataset com m21 para symbolic music, apenas arquivos .krn - symbolic music format
def load_songs_in_kern(dataset_path):
    # percorrer todos os files do dataset e load com music21
    songs = []
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
            if file[-4:] == '.krn':
                song = m21.converter.parse(os.path.join(path,file))
                songs.append(song)
    return songs
# songs = load_songs_in_kern(PATH_erk)
# print(f'Loaded {len(songs)} songs!')

In [ ]:
# verificar durations acceptable
def verify_duration(song, acceptable_durations):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
    return True

#print(f'Has Acceptable Duration? {verify_duration(songs[0],ACCEPTABLE_DURATIONS)}')

# Abre o musescore e mostra a symbolic music
# songs[0].show('midi') to display here on jupyter
#print('Abrindo museScore para testar a musica!')
#songs[0].show('musicxml') # to display on musescore

In [ ]:
#jogar tudo para Cmajor/Aminor para o modelo aprender nessas keys, Caso fosse para as 24 keys exigiria de mais poder computacional
def transpose(song):
    # get the key from song
    parts = song.getElementsByClass(m21.stream.Part) # Part 1 violin, Part 2 cello ....
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4] # index 4 indicando posição de key
    
    # estimate key using m21
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key") # m21 vai estimar qual a key da song caso the key não seja encontrada
    
    print(f'Original Key: {key}')
    
    # get interval for transposition Ex: Bmaj -> Cmaj
    interval = None
    if key.mode == "major":
        print('In case of Major mode map to C')
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        print('In case of Minor mode map to A')
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

    # trnaspose song using calculated interval
    transposed_song = song.transpose(interval)
    
    return transposed_song


#song_0 = transpose(songs[0])
# Abre o musescore e mostra a symbolic music da music 0 transposed
#print('Abrindo museScore para testar a musica transposed!')
#song_0.show('musicxml')

In [ ]:
# encodificar as notas com midi representation numbers, r para rests
def encode_song(song,time_step = 0.25):
    # pitch = 60 , duration 1(quarter note) -> [60, "_", "_", "_"]
    encoded_song = []
    
    for event in song.flat.notesAndRests:
        
        symbol = None
        # preparar encoding notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # midi note ex: 60
        # preparar encoding rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
            
        # convert the note/rest in ts representation -> [60, "_", "_", "_"]
        steps = int(event.duration.quarterLength / time_step) # Evento dura steps - cada step é uma obs da TS
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append('_')
                
    # cast encoded song to a str
    encoded_song = " ".join(map(str,encoded_song))
    
    return encoded_song


In [ ]:
# processar data
def preprocess(dataset_path):
    
    # load the folk songs
    print('#'*100)
    print('Loading songs...')
    songs = load_songs_in_kern(dataset_path)
    print(f'Loaded {len(songs)} songs!')
    
    print('#'*100)
    for index, song in enumerate(songs):
        # filter out songs that have non-acceptable durations
        if verify_duration(song,ACCEPTABLE_DURATIONS) == False:
            # pula current song e vai pra proxima
            continue
            
        
        # transpose songs to Cmaj/Amin
        #todas as keys seram mapeadas para Cmajor em caso de mode == major e Aminor em caso demode = minor
        print(f'Tranposing song {index}')
        song = transpose(song)
        print(f'Done tranposing song {index}')
        print('\n')
        
        
        # encode songs with music time series representation
        print(f'Encoding song {index}')
        encoded_song = encode_song(song)
        print(f'Done encoding song {index}')
        print('\n')
        
        
        # save songs in text file
        print(f'Saving song {index}')
        save_path = os.path.join(SAVE_DIR,str(index))
        with open(save_path,"w") as fp:
            fp.write(encoded_song)
        print('#'*100)

In [ ]:
def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song

def create_single_file_dataset(dataset_path, file_dataset_path, sequence_length):
    
    new_song_delimiter = "/ " * sequence_length
    songs = ""
    
    # load encoded songs and add limiters
    for path, _ , files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            
            songs = songs + song + " " + new_song_delimiter
    
    # eliminando empty string no final 
    songs = songs[:-1]
        
    # save string with all dataset
    with open(file_dataset_path, "w") as fp:
        fp.write(songs)
        
    return songs

In [ ]:
def create_mapping(songs , mapping_path):
    mappings = {}
    
    # identify vocabulary
    songs = songs.split(' ')
    vocab = list(set(songs))
    
    # create mappings
    for index, symbol in enumerate(vocab):
        mappings[symbol] = index
    
    # save vocab to json file
    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp, indent=4)
        
# converter songs to integers
def convert_songs_to_integer(songs):
    
    #load mappings
    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)
    
    #cast songs string to a list
    songs = songs.split()
    
    #map songs int
    int_songs = [mappings[symbol] for symbol in songs]
    
    return int_songs
    
def generate_training_sequences(sequence_length):
    # [11,12,13,14,...] -> input: [11,12], output: 13 ;  input: [12,13], output: 14 ....
    
    # load songs and map to ints
    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_integer(songs)
    
    # generating training sequences
    # 100 symbols, 64 length = 100 - 64 = 36 sequences
    inputs = []
    targets = []
    number_of_sequences = len(int_songs) - sequence_length
    for i in range(number_of_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])
        
    
    # one hot encoding
    # inputs shape = (n of seuqences, sequence lenght, vocab size)
    vocab_size = len(set(int_songs)) # = 18
    inputs = keras.utils.to_categorical(inputs, num_classes=vocab_size)
    targets = np.array(targets)
    
    return inputs, targets

In [ ]:
# celula de teste
def main():
    # pre processar os dados de partituras em strings
    preprocess(PATH_erk)
    
    # todas as mursicas em um arquivo/dataset só
    songs = create_single_file_dataset(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    
    # criando o um dicoinario com representações numerocas para cada elemetno do vocabulario
    create_mapping(songs,MAPPING_PATH)
    
    # entradas e targets da LSTM 
    inputs , targets = generate_training_sequences(SEQUENCE_LENGTH)
    
    print(f'input shape: {inputs.shape} -- targtes shape: {targets.shape}')
    print(f'input type: {type(inputs)} -- targtes type: {type(targets)}')
    
    print(inputs[:1])
    print('#'*100)
    print(targets[:1])
#main()

### Criação e treinamento da LSTM

In [ ]:
#tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
GPU_NAME = tf.test.gpu_device_name()
print(GPU_NAME)


Num GPUs Available:  1
/device:GPU:0


In [ ]:
#load mappings
mappings = None
with open(MAPPING_PATH, "r") as fp:
    mappings = json.load(fp)

# PATH para salvar o modelo
SAVE_MODEL_PATH = '/content/drive/MyDrive/''Musical 2''/projeto/model.h5'

# variaveis globais para LSTM
OUTPUT_UNITS = len(mappings) # num of out put units (classes/labels/notas musicais)

NUM_UNITS = [256] # n of neurons of internal layers , each position correspond to a layer

LOSS = "sparse_categorical_crossentropy" # type of loss function 

LEARNING_RATE = 0.001 # learning rate to apply

EPOCHS = 40
BATCH_SIZE = 64

In [ ]:

def build_model(output_units, num_units, loss, learning_rate):
    
    # create design;architecture of model
    input_layer = keras.layers.Input(shape=(None,output_units))
    lstm_layer = tf.compat.v1.keras.layers.CuDNNLSTM(num_units[0])(input_layer)
    dropout_layer = keras.layers.Dropout(0.2)(lstm_layer)
    output_layer = keras.layers.Dense(output_units, activation='softmax')(dropout_layer)
    
    model = keras.Model(input_layer,output_layer)
    
    # compile
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics = ['accuracy'])
    
    model.summary()
    
    return model
    


def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    print('#'*100)
    print('Iniciando Criação e treinamento da LSTM...')
    
    # generating sequences
    print('#'*100)
    print('Generando sequencias de treinamento...')
    inputs , targets = generate_training_sequences(SEQUENCE_LENGTH)
    
    # build network
    print('#'*100)
    print('Criando Arquitetura do modelo...')
    model = build_model(output_units, num_units, loss, learning_rate)
    
    # train model
    print('#'*100)
    print('Iniciando o Treinamento...')
    # fit model using our gpu
    with tf.device(GPU_NAME):
        model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    
    # save model
    print('#'*100)
    print('Salvando o modelo...')
    model.save(SAVE_MODEL_PATH)
    
    print('#'*100)
    print('DONE')

In [ ]:
train()

####################################################################################################
Iniciando Criação e treinamento da LSTM...
####################################################################################################
Generando sequencias de treinamento...
####################################################################################################
Criando Arquitetura do modelo...
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, 38)]        0         
                                                                 
 cu_dnnlstm_3 (CuDNNLSTM)    (None, 256)               303104    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 3

### Gerador de Melodias

In [ ]:

def sample_with_temperatures(probabilities, temperature):
  # temperature -> infinity - unpredictable choices
  # temperature -> 0 - deterministic
  # temperature -> 1 nothing changes, uses probabilites from model

    predictions = np.log(probabilities) / temperature

    probabilities = np.exp(predictions) / np.sum(np.exp(predictions))

    choices = range(len(probabilities))
    index = np.random.choice(choices, p=probabilities)

    return index

# seed -> melody encoded: '64 _ 54 _ _' ....
# num_steps -> numero de steps que a LSTM vai gerar
# max_sequence_lemgth quantidade maxima de ponto da seed 
# temperature ->   
def generate_melody(seed, num_steps, max_sequence_length, temperature):
    model_path='/content/drive/MyDrive/''Musical 2''/projeto/model.h5'
    model = keras.models.load_model(model_path)

    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)

    start_symbols = ['/'] * SEQUENCE_LENGTH


    # CREATE seed with start symbol
    seed = seed.split()
    melody = seed
    seed = start_symbols + seed

    # map seed to integers
    seed_int = [mappings[symbol] for symbol in seed]


    for _ in range(num_steps):
        # limit seed by max_length
        seed_int = seed_int[-max_sequence_length:]

        # one-hot encode seed
        oneHot_seed = keras.utils.to_categorical(seed_int,num_classes=len(mappings)) # 2 dimensions: (max_sequence_length, num of symbols in vocab)

        # keras expects 3 dimensions so we need to reshape this (max_sequence_length, num of symbols in vocab) for (1 , max_sequence_length, num of symbols in vocab)
        oneHot_seed = oneHot_seed[np.newaxis, ...] 

        # make_predictions
        probabilities = model.predict(oneHot_seed)[0] # [0] pra pegar a probabilidade do primeiro sample(podemos passar batch of samples), resulta = [0.1, 0.3, 0.6]

        output_int = sample_with_temperatures(probabilities, temperature)

        # update seed
        seed_int.append(output_int)

        # map in to our enconding
        output_symbol = [key for key, value in mappings.items() if value == output_int][0]

        # check if we are at the end of a melody
        if output_symbol == '/':
            break

        # update melody
        melody.append(output_symbol)


    return melody

def save_melody(melody, step_duration=0.25, format='midi',file_name="mel.mid"):

    # create music21 stream 
    stream = m21.stream.Stream()

    # parse all the symbols and create rest/notes objects
    start_symbol = None
    step_counter = 1

    for i, symbol in enumerate(melody):
        # handle case in which we have note/rest
        if (symbol != '_') or (i == len(melody)-1):
            # not the initial symbol
            if start_symbol is not None:
                m21_event = None
                quarter_length_duration = step_duration * step_counter
                #rest
                if start_symbol == "r":
                    m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
                #note
                else:
                    m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                stream.append(m21_event)
                # reset step counter
                step_counter = 1

            start_symbol = symbol
            # handle in case of _ (prolongation sign)
        else:
            step_counter += 1

  # write m21 stream to midi file
  stream.write(format,file_name)


### Salvando Melodias
*  **Seed:** melodia encodificada (parte ou completa)
*  **Temperatura:** estrategia de escolhar dentro das probabilidades geradas pela sauda softmax (Ela da maior flexibilidade na criação da nova melodia)

In [ ]:
# testar gerador de melodias
seed = "55 _ 52 _ 55 _ 60 _ 52 _ 55"
temperatures = [0.1 , 0.3, 0.5, 0.7, 0.9]

for temp in temperatures:
    print('#'*100)
    melody = generate_melody(seed, 500, SEQUENCE_LENGTH, temp)
    print(f'Melodia: {melody}')

    temp_value = str(temp).split('.')[-1]
    print(f'Valor da temperatura(sampling): {temp_value}')

    path = '/content/drive/MyDrive/''Musical 2''/projeto/'
    file_name = f'mel_{temp_value}.mid'
    final_path = path + file_name

    save_melody(melody,file_name=final_path)
    print(f'Melodia com temperatura {temp_value} salva!')

####################################################################################################
Melodia: ['55', '_', '52', '_', '55', '_', '60', '_', '52', '_', '55', '_', '60', '_', '60', '_', '59', '_', '57', '_', '57', '_', '55', '_', '55', '_', '55', '_', '52', '_', '_', '_', '55', '_', '_', '_', '60', '_', '_', '_', '64', '_', '_', '_', '62', '_', '_', '_', '_', '_', '60', '_', '59', '_', '57', '_', '55', '_', '_', '_', 'r', '_', '_', '_', '55', '_', '55', '_', '55', '_', '60', '_', '60', '_', '_', '_', '57', '_', '57', '_', '57', '_', '57', '_', '62', '_', '_', '_', '55', '_', '53', '_', '52', '_', '_', '_', '55', '_', '_', '_', '60', '_', '_', '_', '60', '_', '_', '_', '62', '_', '_', '_', '62', '_', '_', '_', '64', '_', '_', '_', '60', '_', '_', '_', 'r', '_', '_', '_', '55', '_', '55', '_', '60', '_', '_', '_', '60', '_', '_', '_', '62', '_', '60', '_', '59', '_', '57', '_', '57', '_', '_', '_', '55', '_', '_', '_', 'r', '_', '_', '_', '55', '_', '_', '_', '55', '_', '_',

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


Melodia: ['55', '_', '52', '_', '55', '_', '60', '_', '52', '_', '55', '_', '60', '_', '60', '_', '59', '_', '57', '_', '57', '_', '55', '_', '55', '_', '55', '_', '60', '_', '_', '_', '59', '_', '60', '_', '62', '_', '_', '_', '64', '_', '_', '_', '60', '_', '_', '_', 'r', '_', '60', '_', '59', '_', '60', '_', '62', '_', '_', '_', '_', '_', '65', '_', '64', '_', '62', '_', '60', '_', '_', '_', 'r', '_', '55', '_', '55', '_', '55', '_', '57', '_', '_', '_', '59', '_', '57', '_', '55', '_', '_', '_', '55', '_', '_', '_', 'r', '_', '55', '_', '55', '_', '64', '_', '62', '_', '_', '_', '_', '_', '60', '_', '59', '_', '57', '_', '57', '_', '_', '_', 'r', '_', '_', '_', '_', '_', '55', '_', '55', '_', '60', '_', '60', '_', '_', '_', 'r', '_', '60', '_', '59', '_', '57', '_', '57', '_', '_', '_', 'r', '_', '_', '_', '55', '_', '_', '_', '55', '_', '_', '_', '60', '_', '_', '_', '55', '_', '_', '_', '52', '_', '_', '_', '55', '_', '_', '_', '60', '_', '_', '_', '_', '_', '_', '_', 'r', '_', '

In [ ]:
# file_name = '/content/drive/MyDrive/''Musical 2''/projeto/mel.mid'
# save_melody(melody,file_name=file_name)